In [6]:
from google.colab import files
uploaded = files.upload()

Saving 연도별 친환경자동차 등록현황.csv to 연도별 친환경자동차 등록현황 (1).csv


In [7]:
from google.colab import files
uploaded = files.upload()

Saving 연료종류별 자동차 등록현황.csv to 연료종류별 자동차 등록현황.csv


In [8]:
from google.colab import files
uploaded = files.upload()

Saving 지역별 주유소 현황.csv to 지역별 주유소 현황.csv


In [14]:
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import YearEnd
from sklearn.linear_model import LinearRegression

############ 옵션 parameter ############
# 예측할 년도 범위
start_year = 2025 # start_year ~ end_year의 데이터 예측 (이상, 이하)
end_year = 2050 # start_year ~ end_year의 데이터 예측 (이상, 이하)
gap_year = 5 # 연 간격

# 주유소 수용률 계산용
gas_gun = 8 # 주유소당 평균 주유기 개수
elec_time = 200 # 한 대의 전기차가 충전을 위해 머무는 시간 (분)
gas_time = 5 # 한 대의 전기차가 충전을 위해 머무는 시간 (분)
#######################################

# CSV 읽기
eco_df = pd.read_csv('연도별 친환경자동차 등록현황.csv', encoding='cp949', index_col=0, header=None, thousands=',').T
car_df = pd.read_csv('연료종류별 자동차 등록현황.csv', encoding='cp949', index_col=0, header=None, thousands=',').T
gas_station_df = pd.read_csv('지역별 주유소 현황.csv', encoding='cp949', index_col=0, header=None, thousands=',').T

year_range = range(start_year, end_year+1, gap_year)
X_predict = pd.DataFrame(data=[datetime.toordinal(datetime(_,12,31)) for _ in year_range], columns=['연도'])

############ 총 차량수 예측 ############
# 연도를 학습에 맞게 ordinal로 변경(데이터 전처리)
car_df['연도'] = pd.to_datetime(car_df['연도'], format='%Y') + YearEnd(0)
car_df['연도'] = car_df['연도'].apply(datetime.toordinal)

# 데이터셋
total_car_X = car_df[['연도']]
total_car_Y = car_df[['차량계']]

# 예측 모델
total_car_lr = LinearRegression()

# 예측 모델 학습
total_car_lr.fit(total_car_X, total_car_Y)

# 예측
total_car_Y_predict = total_car_lr.predict(X_predict).round()

# 0 미만은 0으로 변경
total_car_Y_predict[total_car_Y_predict < 0] = 0

############ 총 전기차수, 친환경차량수 예측 ############
# 연도를 학습에 맞게 ordinal로 변경
eco_df['연도'] = pd.to_datetime(eco_df['연도'], format='%Y') + YearEnd(0)
eco_df['연도'] = eco_df['연도'].apply(datetime.toordinal)

# 데이터셋
eco_X = eco_df[['연도']]
elec_Y = eco_df[['전기차']]
eco_Y = eco_df[['친환경계']]

# 예측 모델
elec_lr = LinearRegression()
eco_lr = LinearRegression()

# 예측 모델 학습
elec_lr.fit(eco_X, elec_Y)
eco_lr.fit(eco_X, eco_Y)

# 예측
elec_Y_predict = elec_lr.predict(X_predict).round()
eco_Y_predict = elec_lr.predict(X_predict).round()

# 0 미만은 0으로 변경
elec_Y_predict[elec_Y_predict < 0] = 0
eco_Y_predict[eco_Y_predict < 0] = 0

############ 총 주유소수 예측 ############
# 연도를 학습에 맞게 ordinal로 변경
gas_station_df['연도'] = pd.to_datetime(gas_station_df['연도'], format='%Y') + YearEnd(0)
gas_station_df['연도'] = gas_station_df['연도'].apply(datetime.toordinal)

# 데이터셋
gas_station_X = gas_station_df[['연도']]
gas_station_Y = gas_station_df[['주유소계']]

# 예측 모델
gas_station_lr = LinearRegression()

# 예측 모델 학습
gas_station_lr.fit(gas_station_X, gas_station_Y)

# 예측
gas_station_Y_predict = gas_station_lr.predict(X_predict).round()

# 0 미만은 0으로 변경
gas_station_Y_predict[gas_station_Y_predict < 0] = 0

############ 주유소 평균 수용률 계산(전국주유소개수 / 전국차량대수)  ############
ratio_df = pd.merge(car_df, gas_station_df[['연도','주유소계']], on ='연도')
gas_car_list = ['휘발유', '경유', 'LPG', '하이브리드', 'CNG']
ratio_df['화석차계'] = ratio_df[gas_car_list].sum(axis=1)
ratio_df['수용률'] = ratio_df['주유소계'] / ratio_df['화석차계'] * gas_gun * elec_time / gas_time

mean_acc = ratio_df["수용률"].mean()

############ 예측 결과 출력 ############
print(f'화석 차량 주유소 평균 수용률 : {mean_acc}')

y_predict_list = [
    ('총 차량 수', total_car_Y_predict.round()),
    ('친환경 자동차 수', eco_Y_predict.round()),
    ('전기 자동차 수', elec_Y_predict.round()),
    ('주유소 수', gas_station_Y_predict.round()),
    ('필요한 전기차 충전기 수', (eco_Y_predict*mean_acc).round())
]

for i, year in enumerate(year_range):
    print(f'\n- {year}년')
    for item in y_predict_list:
        print(f'{item[0]} : {int(item[1][i]):,}')


화석 차량 주유소 평균 수용률 : 0.16204256407658063

- 2025년
총 차량 수 : 27,960,043
친환경 자동차 수 : 196,109
전기 자동차 수 : 196,109
주유소 수 : 10,759
필요한 전기차 충전기 수 : 31,778

- 2030년
총 차량 수 : 31,444,657
친환경 자동차 수 : 287,439
전기 자동차 수 : 287,439
주유소 수 : 9,994
필요한 전기차 충전기 수 : 46,577

- 2035년
총 차량 수 : 34,929,270
친환경 자동차 수 : 378,770
전기 자동차 수 : 378,770
주유소 수 : 9,229
필요한 전기차 충전기 수 : 61,377

- 2040년
총 차량 수 : 38,415,791
친환경 자동차 수 : 470,151
전기 자동차 수 : 470,151
주유소 수 : 8,463
필요한 전기차 충전기 수 : 76,184

- 2045년
총 차량 수 : 41,900,404
친환경 자동차 수 : 561,482
전기 자동차 수 : 561,482
주유소 수 : 7,698
필요한 전기차 충전기 수 : 90,984

- 2050년
총 차량 수 : 45,385,017
친환경 자동차 수 : 652,812
전기 자동차 수 : 652,812
주유소 수 : 6,933
필요한 전기차 충전기 수 : 105,783
